In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import gc
import numpy as np
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers.dict_nbhd_transformer import DictNBHDTransformer
from transformers.dict_transformer import DictTransformer
from transformers.flat_transformer import FlatTransformer
from transformers.dict_class_transformer import DictClassTransformer
from transformers.digit_transformer import DigitTransformer
from transformers.dash_transformer import DashTransformer
from transformers.context_extractor import ContextExtractor
from sklearn.pipeline import Pipeline

In [2]:
INPUT_PATH = r'../input/norm_challenge_ru'
DATA_INPUT_PATH = r'../input/norm_challenge_ru/ru_with_types'
SUBM_PATH = INPUT_PATH

df = pd.read_csv(os.path.join(INPUT_PATH, 'ru_train.csv'), encoding='utf-8', index_col=False)
# df['before'] = df['before'].str.lower()
# df['after'] = df['after'].str.lower()
df['before_prevprev'] = df['before'].shift(2)
df['before_prev'] = df['before'].shift(1)
df['before_next'] = df['before'].shift(-1)
df['before_nextnext'] = df['before'].shift(-2)
df = df.fillna('')

print(df.info())

classes = ['PLAIN', 'DATE', 'PUNCT', 'ORDINAL', 'VERBATIM', 'LETTERS', 'CARDINAL', 'MEASURE', 'TELEPHONE', 'ELECTRONIC', 'DECIMAL', 'DIGIT', 'FRACTION', 'MONEY', 'TIME']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10574516 entries, 0 to 10574515
Data columns (total 9 columns):
sentence_id        int64
token_id           int64
class              object
before             object
after              object
before_prevprev    object
before_prev        object
before_next        object
before_nextnext    object
dtypes: int64(2), object(7)
memory usage: 726.1+ MB
None


In [3]:
ce = ContextExtractor()
list(ce.transform(df['before']))

ContextExtractor fit:   0%|          | 0/10574516 [00:00<?, ?it/s]

ContextExtractor fit:   0%|          | 1370/10574516 [00:00<12:52, 13695.47it/s]

ContextExtractor fit:   0%|          | 2482/10574516 [00:00<14:12, 12404.79it/s]

ContextExtractor fit:   0%|          | 3348/10574516 [00:00<15:48, 11148.86it/s]

ContextExtractor fit:   0%|          | 4088/10574516 [00:00<17:15, 10209.46it/s]

...

ContextExtractor fit: 100%|█████████▉| 10571002/10574516 [13:12<00:00, 13330.78it/s]

ContextExtractor fit: 100%|█████████▉| 10572460/10574516 [13:13<00:00, 13330.83it/s]

ContextExtractor fit: 100%|█████████▉| 10573890/10574516 [13:13<00:00, 13329.74it/s]

ContextExtractor fit: 100%|██████████| 10574516/10574516 [13:13<00:00, 13329.76it/s]

[(True,
  False,
  'PREP',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (False,
  False,
  'NOUN',
  'inan',
  None,
  'datv',
  'neut',
  None,
  None,
  'sing',
  None,
  None,
  None,
  None),
 (False,
  False,
  'PREP',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (False,
  False,
  'NOUN',
  'inan',
  None,
  'nomn',
  'masc',
  None,
  None,
  'sing',
  None,
  None,
  None,
  None),
 (False,
  False,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (True,
  False,
  'VERB',
  None,
  'impf',
  None,
  None,
  None,
  'indc',
  'plur',
  None,
  'past',
  'intr',
  None),
 (False,
  False,
  'ADJF',
  None,
  None,
  'ablt',
  None,
  None,
  None,
  'plur',
  None,
  None,
  None,
  None),
 (False,
  False,
  'NOUN',
  'inan',
  None,
  'ablt',
  'femn',
  None,
  None,
  'plur',
  None,
  None,
  None,
  None),
 (False,
  False,
  'CONJ',
  

In [4]:
x_train, x_valid, y_train, y_valid = train_test_split(df.drop(['after'], axis=1), df['after'], test_size=0.1)

In [5]:
pipeline = Pipeline([
    ('dash', DashTransformer()),
    ('digit', DigitTransformer()),
    ('punсt', DictClassTransformer(u'PUNCT', 1.0)),
    ('verbatim', DictClassTransformer(u'VERBATIM', 1.0)),
    ('dict_nbhd', DictNBHDTransformer(1.0)),
    ('dict', DictTransformer(1.0)),
    ('flat', FlatTransformer())
])
predicted_val = pipeline.fit_transform(x_train, y_train)
print(f'pipeline val TP {np.mean(predicted_val["after"] == y_train)}', flush=True)
print(f'pipeline val FP {np.mean(~(predicted_val["after"] == y_train) & ~(predicted_val["after"].isnull()))}', flush=True)
predicted = pipeline.transform(x_valid)
print(f'pipeline test TP {np.mean(predicted["after"] == y_valid)}', flush=True)
print(f'pipeline test FP {np.mean(~(predicted["after"] == y_valid) & ~(predicted["after"].isnull()))}', flush=True)

DashTransformer transform:   0%|          | 0/9517064 [00:00<?, ?it/s]

DashTransformer transform:   0%|          | 17708/9517064 [00:00<00:53, 177019.82it/s]

DashTransformer transform:   0%|          | 37650/9517064 [00:00<00:50, 188062.76it/s]

DashTransformer transform:   1%|          | 79313/9517064 [00:00<00:35, 264344.15it/s]

DashTransformer transform:   2%|▏         | 151982/9517064 [00:00<00:24, 379887.43it/s]

...

DictTransformer transform:  88%|████████▊ | 929575/1057452 [00:01<00:00, 619693.91it/s]

DictTransformer transform:  94%|█████████▍| 995598/1057452 [00:01<00:00, 622225.89it/s]

DictTransformer transform: 100%|██████████| 1057452/1057452 [00:01<00:00, 624225.33it/s]

pipeline test TP 0.9646111596554737


pipeline test FP 0.03538884034452627


In [12]:
predicted[(predicted["after"] == y_valid)]['class'].value_counts()


PLAIN    562
Name: class, dtype: int64

In [13]:
predicted[~(predicted["after"] == y_valid) & ~(predicted["after"].isnull())]['class'].value_counts()

VERBATIM    26
Name: class, dtype: int64

In [14]:
predicted[(predicted["after"].isnull())]['class'].value_counts()

PLAIN         736028
PUNCT         228530
CARDINAL       27146
LETTERS        18831
DATE           18629
VERBATIM       15838
ORDINAL         4624
MEASURE         4013
TELEPHONE        975
DECIMAL          747
ELECTRONIC       604
FRACTION         252
MONEY            243
TIME             203
DIGIT            201
Name: class, dtype: int64

In [16]:
predicted[~(predicted["after"] == y_valid) & ~(predicted["after"].isnull()) & (predicted['class']=='VERBATIM')].assign(after_true=y_valid)

,sentence_id,token_id,class,before,before_prev,before_next,after,after_true
2898416,209803,1,VERBATIM,-,2002,03,до,-
8604929,619877,21,VERBATIM,-,1882,9,до,-
5062798,365653,3,VERBATIM,-,2002,04,до,-
9830359,708042,15,VERBATIM,-,480,221 до н.э.,до,-
9672470,696701,8,VERBATIM,-,10,009,до,-
2617457,189523,2,VERBATIM,-,64,0702,до,-
1643002,119246,3,VERBATIM,-,2003,02,до,-
2586103,187220,3,VERBATIM,-,1951,02,до,-
1659127,120385,22,VERBATIM,-,1914,08.04.1915,до,-
8603152,619742,20,VERBATIM,-,1999,03,до,-


In [50]:
df[~(df['before'].str.match('^-?\d*\s?\d*\s?\/\s?\d+\s?\d*$')) & (df['class'] == 'FRACTION')]

,sentence_id,token_id,class,before,after,before_prev,before_next
40420,2962,7,FRACTION,7½,семи целых и одной второй,около,фунтов
157446,11425,6,FRACTION,3½,три целых и одну вторую,на,очка
445580,32294,9,FRACTION,59½,пятидесяти девяти целых и одной второй,139 976 рублей,копейки
765884,55575,1,FRACTION,¼,одна четвертая,второй,финалдата
803808,58316,6,FRACTION,½,одна вторая,номиналом,шиллинга
999135,72597,18,FRACTION,⅔,две третьих,на,территории
1253476,91132,28,FRACTION,½,одна вторая,cra,gut
1584453,115013,1,FRACTION,8½,восемь целых и одна вторая,«,женщин
1584466,115013,14,FRACTION,8½,восемь целых и одна вторая,.,women
1876892,136047,6,FRACTION,½,одна вторая,в,","
